# 3D Printing Precision Parts
## Problem Definition
You work for a company that manufactures high quality polymer parts for OrgoCorp, a galactic bioengineering company. The quality requirements are so high that you need to use very sophisticated 3D printing machines. You need to develop a mathematical programming model to organize monthly production considering:

- **Fixed operational costs (euros):** Each 3D printing machine $m$ has a different fixed operational costs $Fm$ that needs to be considered when a printing machine is used to print any units.
- **Unitary production costs (euros):** Each 3D printing machine $m$ produces parts of product type $p$ at a different unitary costs $C_{mp}$
- **Capacity (minutes)**: Each 3D printing machine $m$ has a different capacity $S_m$ (minutes)
- **Speed (minutes/part)**: Each 3D printing machine $m$ prints a unit of product type $p$ at a different unitary speed
- **Demand (units)**: OrgoCorp has confirmed a demand for $d_p$ units of every product type $p$
Build a model that takes into account these requirements, identifying indices, decision variables, objective function and constraints.

The CEO of OrgoCorp, The High Evolutionary, would like to set up a new contract allowing more flexibility in the delivery of units, such that:

- **Demand per period (units)**: Now you need to consider that the demand for every product type $p$ is not constant and depends on the planning period $t$, $D_{pt}$
- **Delayed demand (units)**: You may delay the production for one period. For every unit of product type $p$ that is delayed, OrgoCorp will charge an extra delayed demand cost of $B_{pt}$
Modify the model to take into account this new contract

## Solution
### Part I: No delayed demand
The following mathematical programming model is proposed to model the problem:

#### Indices

- $m \in M$: Set of 3D printing machines
- $p \in P$: Set of product types

#### Decision Variables

- $x_{mp}$: (Integer) Number of units of product type $p$ printed by 3D printing machine $m$
- $Y_{m}$: (Binary) 1 if 3D printing machine $m$ is used, 0 otherwise

#### Objective Function

- **Minimize** the total cost of production

$$\min z = \sum_{m=1}^{M}{F_m·Y_m} + \sum_{m=1}^{M}{\sum_{p=1}^{P}{C_{mp}·x_{mp}}}$$

where the first term are the fixed operational costs and the second term are the unitary production costs.

#### Constraints
- **Capacity constraint:** The total time used by all the 3D printing machines cannot exceed the total capacity of the machines

$$\sum_{p=1}^{P}{x_{mp}·T_{mp}} \leq S_m·Y_m \quad \forall m \in [1, 2, ..., M]$$

Where $T_{mp}$ is the time needed (in minutes) to print a unit of product type $p$ in 3D printing machine $m$. Based on the data provided, this time is calculated as the inverse of the speed of the machine for that product type, which we can note as $S_{mp}$, that is $T_{mp} = \frac{1}{S_{mp}}$

Note that we are using the binary variable $Y_m$ to ensure that the capacity constraint is only considered if the machine is used.

- **Demand constraint:** The total number of units of product type $p$ printed by all the 3D printing machines must be equal or greater than the demand for that product type

$$\sum_{m=1}^{M}{x_{mp}} = d_p \quad \forall p \in [1, 2, ..., P]$$

That is, the total number of units of product type $p$ printed by all the 3D printing machines must be equal to the demand for that product type. We could add some slack to this constraint, making the demand constraints of type greater or equal, but we are assuming that the demand is always met exactly (being a precision parts company, we cannot afford to deliver more or less units than the ones requested).

#### Model
Finally, the complete model is:

$$\min z = \sum_{m=1}^{M}{F_m·Y_m} + \sum_{m=1}^{M}{\sum_{p=1}^{P}{C_{mp}·x_{mp}}}$$

$$\text{s.t}$$

$$\sum_{p=1}^{P}{x_{mp}·T_{mp}} \leq S_m·Y_m \quad \forall m \in [1, 2, ..., M]$$

$$\sum_{m=1}^{M}{x_{mp}} = d_p \quad \forall p \in [1, 2, ..., P]$$


### Part II: Delayed demand
Based on the model proposed in part I, we need to introduce a new index $t$ to represent the planning period. We also need to modify the decision variables to introduce the new index and allow our solution to adapt production in each period to demand. Therefore, the decision variables $x_{mp}$ now becomes $x_{mpt}$, and $Y_{m}$ becomes $Y_{mt}$. We also need to introduce a new set of decision variables to model delayed demand, $D_{mpt}$, which represents the number of units of product type $p$ that are delayed in period $t$ and printed in period $t+1$ by 3D printing machine $m$, and modify the objective function and constraints accordingly.

#### Indices
- $m \in M$: Set of 3D printing machines
- $p \in P$: Set of product types
- $t \in T$: Set of planning periods

#### Decision Variables
- $x_{mpt}$: (Integer) Number of units of product type $p$ printed by 3D printing machine $m$ in period $t$
- $Y_{mt}$: (Binary) 1 if 3D printing machine $m$ is used in period $t$, 0 otherwise
- $I_{pt}$: (Integer) Number of units of product type $p$ delayed in period $t$ and printed in period $t+1$

#### Objective Function
- **Minimize** the total cost of production
$$\min z = \sum_{t=1}^{T}{\sum_{m=1}^{M}{F_m·Y_{mt}}} + \sum_{t=1}^{T}{\sum_{m=1}^{M}{\sum_{p=1}^{P}{C_{mp}·x_{mpt}}}} + \sum_{t=1}^{T}{\sum_{p=1}^{P}{B_{pt}·I_{pt}}}$$

where the first term are the fixed operational costs, the second term are the unitary production costs and the third term are the delayed demand costs.

#### Constraints
- **Capacity constraint:** The total time used by all the 3D printing machines cannot exceed the total capacity of the machines. We need to consider the time period:

$$\sum_{p=1}^{P}{x_{mpt}·T_{mp}} \leq S_m·Y_{mt} \quad \forall m \in [1, 2, ..., M], \forall t \in [1, 2, ..., T]$$

- **Demand constraint:** The total number of units of product type $p$ printed by all the 3D printing machines must be equal or greater than the demand for that product type. Now, we need to make sure that the units produced satisfy the demand and the delayed demand:

$$\sum_{m=1}^{M}{x_{mpt}} = I_{pt-1} + D_{pt} - I_{pt} \quad \forall p \in [1, 2, ..., P], \forall t \in [1, 2, ..., T]$$

$$I_{p0} = 0 \quad \forall p \in [1, 2, ..., P]$$

That is, the number of units produced in period $t$ must be equal to the units delayed in the previous period $t-1$ plus the demand in period $t$ minus the units delayed for the next period $t+1$. Note that we are considering that the units delayed in period $t$ are printed in period $t+1$, so we need to consider the units delayed in period $t-1$ to calculate the units produced in period $t$. Finally, we are considering that there are no units delayed in the previous period $t=0$.

#### Model
Summarizing, the complete model is:
$$\min z = \sum_{t=1}^{T}{\sum_{m=1}^{M}{F_m·Y_{mt}}} + \sum_{t=1}^{T}{\sum_{m=1}^{M}{\sum_{p=1}^{P}{C_{mp}·x_{mpt}}}} + \sum_{t=1}^{T}{\sum_{p=1}^{P}{B_{pt}·I_{pt}}}$$

$$\text{s.t}$$

$$\sum_{p=1}^{P}{x_{mpt}·T_{mp}} \leq S_m·Y_{mt} \quad \forall m \in [1, 2, ..., M], \forall t \in [1, 2, ..., T]$$

$$\sum_{m=1}^{M}{x_{mpt}} = I_{pt-1} + D_{pt} - I_{pt} \quad \forall p \in [1, 2, ..., P], \forall t \in [1, 2, ..., T]$$

$$I_{p0} = 0 \quad \forall p \in [1, 2, ..., P]$$






